In [2]:
from alfrd.lib import GSC, LogFrame
from alfrd.util import timeinmin, read_inputfile
from pathlib import Path
import time, subprocess
import glob

### Example 1 : Initializing and creating instance

In [3]:
url='https://docs.google.com/spreadsheets/d/1gywTLuGxkTfX-nIwUHynR4wvLNv-NOwJeykqw0ThPT0/edit#gid=0'
worksheet='main'
gsc = GSC(url=url, wname=worksheet, key='path/to/json/file')      # default path for key = home/usr/.alfred
_ = gsc.open()                               

Success!


In [13]:
wid = 0                                                                         #   (default)
gsc = GSC(sid='1gyadsuadsTfX-adsUHynR4wvLNv-NOadsykqw0ThREW', wid=wid)          #   either give the url or the spreadsheet id
_ = gsc.open()

Success!


### Example 2: Inititalize the framework

In [4]:
lf = LogFrame(gsc)
lf.primary_colname          =   'FILE_NAME'

### Example 3: Run - Iterate for each row

In [5]:
count,failed=0,0
allfiles=[]
folder_for_fits ='path/to/allfits'

allfiles.extend(glob.glob(f"{folder_for_fits}*/*fits"))

for fitsfile in allfiles:
    fitsfile_name = Path(fitsfile).name
    lf.primary_value            =   fitsfile_name

### Example 4: using column logic and adding values on success/failures

In [6]:
if lf.isvalue(value='True', colname='TSYS') and lf.isval_unique('Project'):
    
    lf.working_col          =   'Comment1'                                          # working column
    print(lf.get_value())

    result                  =   'test'
    if result :
        count               =   lf.put_value(result, count=count)                   
    else:
        failed              =   lf.put_value('failed: logfile_path', count=failed)
    
    print(lf.get_value())

test
test


### Example 5: Script execution

In [7]:
def run_picard(cmd):
    t0 = time.time()
    subprocess.run(cmd)
    t1 = time.time()
    td = timeinmin(t1-t0)
    return td

def scripted_picard_1(wd_ifolder, count, failed, col='fits to ms'):
    """
    converts fitsfile to ms file; checks if conversion was successful; skips if file exists; logs in the spreadsheet;
    """
    td=0
    cmd=["picard",'-n','10',"-l","e",'--input',wd_ifolder]
       
    params, files, input_folder = read_inputfile(wd_ifolder, "observation.inp")
    
    if not Path(f"{wd_ifolder}").exists():
        print('input folder missing..')
    elif not Path(f"{wd_ifolder}/../{params['ms_name']}").exists():
        td          =   run_picard(cmd)
        count       =   lf.put_value(colname=col, value=td, count=count)
    elif Path(f"{wd_ifolder}/../{params['ms_name']}").exists():
        print('skipped')

    if not Path(f"{wd_ifolder}/../{params['ms_name']}").exists():
        count-=1
        failed      =   lf.put_value(colname=col, value='failed',count=failed)
    return td, count, failed

if lf.isvalue(value='True', colname='TSYS') and lf.isval_unique('Project'):
    lf.working_col          =   'Comment1'
    ttaken, count, failed   =   scripted_picard_1(wd_ifolder='path/to/wd/input_template', count=count, failed=failed )

print(lf.get_value(colname='fits to ms'))

input folder missing..
failed


### Example 6: Update the sheet

In [14]:
lf.update_sheet(count=count, failed=failed, csvfile='df_sheet.csv')                     # if updating the sheet fails, a copy of the dataframe is saved locally at the csvfile path.

Updated!


In [ ]:
# can also update from a dataframe object/ by reading from a csv file
from pandas import read_csv
lf.df_sheet = read_csv('df_sheet.csv')

### Example 7: Conditional Formatting

need to run only once.

In [8]:
r1 = lf.create_rule(range='G2:G105', type='TEXT_CONTAINS' ,value='False', c='r')               # check if TSYS == False --> background color = red
r2 = lf.create_rule(range='G2:G105', type="TEXT_CONTAINS", value='True', custom_clr=lf.create_color(0.56, 0.77, 0.49))
r3 = lf.create_conditional_format(range='F2:F105', c='g', valtype='timeinmin')              # check if value in XXmYYs format --> background color = green

r4 = lf.create_conditional_format(range='F2:F105', c='r', valtype='fail')                   # check if value contains fail --> background color = red
r4 = lf.create_rule(range='F2:F105', type='TEXT_CONTAINS', c='r', value='fail')                # similar to above
print(r1,r2,r3,r4)
lf.add_conditional_format(r1, r2, r3, r4)

ranges=[<GridRange sheetId=0;startRowIndex=1;endRowIndex=105;startColumnIndex=6;endColumnIndex=7>];booleanRule=(condition=(type=TEXT_CONTAINS;values=[<ConditionValue userEnteredValue=False>]);format=(backgroundColor=(red=0.8784314;green=0.4;blue=0.4))) ranges=[<GridRange sheetId=0;startRowIndex=1;endRowIndex=105;startColumnIndex=6;endColumnIndex=7>];booleanRule=(condition=(type=TEXT_CONTAINS;values=[<ConditionValue userEnteredValue=True>]);format=(backgroundColor=(red=0.56;green=0.77;blue=0.49))) ranges=[<GridRange sheetId=0;startRowIndex=1;endRowIndex=105;startColumnIndex=5;endColumnIndex=6>];booleanRule=(condition=(type=CUSTOM_FORMULA;values=[<ConditionValue userEnteredValue==AND(ISNUMBER(SEARCH("m", F2:F105)), ISNUMBER(SEARCH("s", F2:F105)))>]);format=(backgroundColor=(red=0.56;green=0.77;blue=0.49))) ranges=[<GridRange sheetId=0;startRowIndex=1;endRowIndex=105;startColumnIndex=5;endColumnIndex=6>];booleanRule=(condition=(type=TEXT_CONTAINS;values=[<ConditionValue userEnteredValue=f